# Mixing `ipywidgets` and `bokeh` plots


## About this notebook

This notebook belongs to a series of small projects which aim is to evaluate the [Jupyter](http://jupyter.org/) ecosystem for science experiments control. The main idea is to use the _Juypter notebook_ as a convergence platform in order to offer a fully featured environment to scientists. 

## About bokeh

Experiments control requires both static and dynamic (i.e live) data visualization. Since Jupyter doesn't provide any 'official' data visualization solution, we need to select one. Among the available solutions, [bokeh](http://bokeh.pydata.org/en/latest) presents the highest potential for our application.

Bokeh as been selected for its:
1. [built-in notebook integration](http://bokeh.pydata.org/en/latest/docs/user_guide/notebook.html)
2. built-in [data streaming](http://bokeh.pydata.org/en/latest/docs/reference/models/sources.html#bokeh.models.sources.ColumnDataSource.patch) [features](http://bokeh.pydata.org/en/latest/docs/reference/models/sources.html#bokeh.models.sources.ColumnDataSource.stream) for live plots update 
3. ability to add [custom or specialized behaviors](http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html) in response to property changes and other events
4. [graphics quality](http://bokeh.pydata.org/en/latest/docs/gallery.html#gallery)

Have a look to this [quickstart](http://bokeh.pydata.org/en/latest/docs/user_guide/quickstart.html) for a bokeh overview.

## Topic of the day

Mix ipywidgets and bokeh plots in a jupyter notebook and see what's happen. 

See also [this notebook](https://github.com/nleclercq/jupyter-for-controls/blob/master/bokeh-data-streaming-for-notebook/b4nb.ipynb) for another ipywidgets + bokeh example.

### First, we route bokeh outputs to notebook cells
This will also load BokehJS - the JavaScript part of bokeh.

In [1]:
from bokeh.io import output_notebook 
output_notebook()

Loading BokehJS ...

### LivePlot class

In [2]:
import numpy as np
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource

from IPython.display import clear_output, display

class LivePlot(object):
    
    def __init__(self, doc):
        self._doc = doc
        self._np = 500
        self._callback_period = 1.
        columns = dict()
        columns['x'] = self.__gen_x_scale()
        columns['y'] = self.__gen_y_random_data()
        self._cds = ColumnDataSource(data=columns)
        self.show()
        
    def __gen_x_scale(self):
        """x data"""
        return np.linspace(1, self._np, num=self._np, endpoint=True)
    
    def __gen_y_random_data(self):
        """y data"""
        return np.random.rand(self._np)

    def show(self):
        f = figure(plot_width=950, plot_height=200)
        f.toolbar_location = 'above'
        f.line(x='x', y='y', source=self._cds, color="navy", alpha=0.5)
        self.__periodic_callback = self._doc.add_periodic_callback(self.__periodic_job, 1000. * self._callback_period)
        self._doc.add_root(f)

    @property
    def num_points(self):
        return self._np
     
    @num_points.setter
    def num_points(self, new_value):
        self._np = int(new_value)
    
    @property
    def callback_period(self):
        return self._callback_period
     
    @callback_period.setter
    def callback_period(self, new_value):
        self._callback_period = new_value
        self._doc.remove_periodic_callback(self.__periodic_callback)
        self.__periodic_callback = self._doc.add_periodic_callback(self.__periodic_job, 1000. * self._callback_period)
        
    def __periodic_job(self):
        self._cds.data.update(x=self.__gen_x_scale(), y=self.__gen_y_random_data())
        
    def close(self):
        self._doc.add_next_tick_callback(self.__safe_close)
   
    def __safe_close(self):
        self._doc.remove_periodic_callback(self.__periodic_callback)
        self._doc.clear()
        clear_output()

### LivePlotController class

In [3]:
import ipywidgets as widgets

class LivePlotController(object):
    """a LivePlot controller using ipywidgets"""

    def __init__(self, lp):
        self._live_plot = lp
        self._controls = None
        self.__setup_controls()
        
    @staticmethod
    def l01a(width='auto', *args, **kwargs):
        return widgets.Layout(flex='0 1 auto', width=width, *args, **kwargs)

    @staticmethod
    def l11a(width='auto', *args, **kwargs):
        return widgets.Layout(flex='1 1 auto', width=width, *args, **kwargs)

    def __setup_num_point_slider(self):
        return widgets.IntSlider(
            value=self._live_plot.num_points,
            min=0,
            max=1000,
            step=10,
            description='Num. points',
            disabled=False,
            continuous_update=True,
            orientation='horizontal',
            readout=True
        )

    def __setup_update_period_slider(self):
        return widgets.FloatSlider(
            value=self._live_plot.callback_period,
            min=0.1,
            max=2,
            step=0.1,
            description='Update Period [s]',
            disabled=False,
            continuous_update=True,
            orientation='horizontal',
            readout=True,
            readout_format='.2f',
            style={'description_width':'initial'}
        )
    
    def __setup_controls(self, **kwargs):
        np_slider = self.__setup_num_point_slider()
        np_slider.observe(self.__on_num_points_changed, names='value')
        up_slider = self.__setup_update_period_slider()
        up_slider.observe(self.__on_update_period_changed, names='value')
        close_button = widgets.Button(description="Close", layout=self.l01a(width="100px"))
        close_button.on_click(self.__on_close_clicked)
        self._controls = widgets.HBox([np_slider, up_slider, close_button], layout=self.l01a())
        display(self._controls)
        
    def __on_num_points_changed(self, event):
        try:
            #print("__on_refresh_period_changed: {}".format(event))
            self._live_plot.num_points = event['new']
        except Exception as e:
            print(e)

    def __on_update_period_changed(self, event):
        try:
            #print("__on_update_period_changed: {}".format(event))
            self._live_plot.callback_period = event['new']
        except Exception as e:
            print(e)
            
    def __on_close_clicked(self, b=None):
        self._controls.close()
        self._live_plot.close()

### Embedded Bokeh Server

In [6]:
import logging
from bokeh.plotting import show
from bokeh.application import Application
from bokeh.application.handlers.function import FunctionHandler

logging.getLogger('bokeh.server.server').setLevel(logging.ERROR)

def setup_document(doc):
    LivePlotController(LivePlot(doc))
    return doc
      
app = Application(FunctionHandler(setup_document))

show(app)